In [ ]:
# import torch
# from transformers import LlamaForCausalLM, LlamaTokenizer

# # Load the model and tokenizer
# model_name = "facebook/llama-2"  # Replace with the correct model name
# tokenizer = LlamaTokenizer.from_pretrained(model_name)
# model = LlamaForCausalLM.from_pretrained(model_name)

# # Ensure the model is on the correct device (GPU if available)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # Function to evaluate the model on a given prompt
# def evaluate_model(prompt, max_length=50):
#     inputs = tokenizer(prompt, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model.generate(
#             inputs.input_ids,
#             max_length=max_length,
#             num_return_sequences=1,
#             no_repeat_ngram_size=2,
#             do_sample=True,
#             top_k=50,
#             top_p=0.95
#         )
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# # Example prompt
# prompt = "Once upon a time"

# # Evaluate the model
# output = evaluate_model(prompt)
# print("Generated text:", output)


# Setup

In [ ]:
%%capture
%pip install git+https://github.com/neelnanda-io/TransformerLens.git

In [ ]:
import torch
import numpy as np
from pathlib import Path

In [ ]:
# import transformer_lens
import transformer_lens.utils as utils
# from transformer_lens.hook_points import (
#     HookedRootModule,
#     HookPoint,
# )  # Hooking utilities
# from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

# Load model

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
LLAMA_2_7B_CHAT_PATH = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(LLAMA_2_7B_CHAT_PATH)
hf_model = LlamaForCausalLM.from_pretrained(LLAMA_2_7B_CHAT_PATH, low_cpu_mem_usage=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
import transformer_lens.utils as utils
from transformer_lens.hook_points import HookPoint
from transformer_lens import HookedTransformer

In [ ]:
# Get list of arguments to pass to `generate` (specifically these are the ones relating to sampling)
generate_kwargs = dict(
    do_sample = False, # deterministic output so we can compare it to the HF model
    top_p = 1.0, # suppresses annoying output errors
    temperature = 1.0, # suppresses annoying output errors
)

In [ ]:
tl_model = HookedTransformer.from_pretrained(
    LLAMA_2_7B_CHAT_PATH,
    hf_model = hf_model,
    tokenizer = tokenizer,
    device = "cpu",
    fold_ln = False,
    center_writing_weights = False,
    center_unembed = False,
)

del hf_model

tl_model = tl_model.to("cuda" if torch.cuda.is_available() else "cpu")

Loaded pretrained model meta-llama/Llama-2-7b-chat-hf into HookedTransformer
Moving model to device:  cuda


# spanish

In [ ]:
prompt = "uno"
output = tl_model.generate(prompt, max_new_tokens=1, **generate_kwargs)
print(output)

  0%|          | 0/1 [00:00<?, ?it/s]

unoconv


In [ ]:
prompt = "uno dos"
output = tl_model.generate(prompt, max_new_tokens=1, **generate_kwargs)
print(output)

  0%|          | 0/1 [00:00<?, ?it/s]

uno dos tres


In [ ]:
prompt = "uno dos tres"
output = tl_model.generate(prompt, max_new_tokens=1, **generate_kwargs)
print(output)

  0%|          | 0/1 [00:00<?, ?it/s]

uno dos tres cuatro


In [ ]:
prompt = "uno dos tres cuatro cinco seis"
output = tl_model.generate(prompt, max_new_tokens=1, **generate_kwargs)
print(output)

  0%|          | 0/1 [00:00<?, ?it/s]

uno dos tres cuatro cinco seis seven


In [ ]:
prompt = "uno mas tres"
output = tl_model.generate(prompt, max_new_tokens=2, **generate_kwargs)
print(output)

  0%|          | 0/2 [00:00<?, ?it/s]

uno mas tres 




In [ ]:
prompt = "uno tres cinco"
output = tl_model.generate(prompt, max_new_tokens=2, **generate_kwargs)
print(output)

  0%|          | 0/2 [00:00<?, ?it/s]

uno tres cinco




# German

In [ ]:
prompt = "eins zwei drei"
output = tl_model.generate(prompt, max_new_tokens=5, **generate_kwargs)
print(output)

  0%|          | 0/5 [00:00<?, ?it/s]

eins zwei drei


```



In [ ]:
prompt = "eins"
output = tl_model.generate(prompt, max_new_tokens=5, **generate_kwargs)
print(output)

  0%|          | 0/5 [00:00<?, ?it/s]

einsiedel.com



# French

In [ ]:
prompt = "un, deux, trois"
output = tl_model.generate(prompt, max_new_tokens=5, **generate_kwargs)
print(output)

  0%|          | 0/5 [00:00<?, ?it/s]

un, deux, trois, four, five,


# math reasoning

In [ ]:
prompt = "Bob is three years older than Steve. If Bob is 6, how old is Steve?"
output = tl_model.generate(prompt, max_new_tokens=50, **generate_kwargs)
print(output)

  0%|          | 0/50 [00:00<?, ?it/s]

Bob is three years older than Steve. If Bob is 6, how old is Steve?

Answer: If Bob is 6 years old and Bob is 3 years older than Steve, then Steve is 6 - 3 = 3 years old.</s>


In [ ]:
prompt = "Bob is three years older than Steve. If Bob is 6 then Steve is"
output = tl_model.generate(prompt, max_new_tokens=50, **generate_kwargs)
print(output)

  0%|          | 0/50 [00:00<?, ?it/s]

Bob is three years older than Steve. If Bob is 6 then Steve is 3.

Answer:
Bob is 6 years old, and Steve is 3 years old.</s>


# Days of week

In [ ]:
prompt = "Today is Monday. What day comes next?"
output = tl_model.generate(prompt, max_new_tokens=50, **generate_kwargs)
print(output)

  0%|          | 0/50 [00:00<?, ?it/s]

Today is Monday. What day comes next?

Answer: Tuesday.</s>


In [ ]:
prompt = "Today is Monday. In two days, it will be"
output = tl_model.generate(prompt, max_new_tokens=50, **generate_kwargs)
print(output)

  0%|          | 0/50 [00:00<?, ?it/s]

Today is Monday. In two days, it will be Wednesday. In three days, it will be Friday. In four days, it will be Saturday. In five days, it will be Sunday.

What day is today?

Answer: Today is Monday.</s>


# Distinguish Additive vs Multiplicative

In [ ]:
prompt =  "2 4 6 8 10"
output = tl_model.generate(prompt, max_new_tokens=10, **generate_kwargs)
print(output)

  0%|          | 0/10 [00:00<?, ?it/s]

2 4 6 8 10 12 14 16 


In [ ]:
prompt =  "2 4 8 16 32"
output = tl_model.generate(prompt, max_new_tokens=8, **generate_kwargs)
print(output)

  0%|          | 0/8 [00:00<?, ?it/s]

2 4 8 16 32 64 128



In [ ]:
prompt =  "3 6 9 12 15"
output = tl_model.generate(prompt, max_new_tokens=15, **generate_kwargs)
print(output)

  0%|          | 0/15 [00:00<?, ?it/s]

3 6 9 12 15 18 21 24 27 30


# less than

In [ ]:
prompt =  "Bob is older than Steve. Bob is age 10 and Steve is age"
output = tl_model.generate(prompt, max_new_tokens=100, **generate_kwargs)
print(output)

  0%|          | 0/100 [00:00<?, ?it/s]

Bob is older than Steve. Bob is age 10 and Steve is age 8.

Answer: Bob is older than Steve.

Explanation:
Bob is 10 years old, and Steve is 8 years old. Bob is 2 years older than Steve.</s>


# Fibonacci

In [ ]:
prompt =  "0 1 1 2 3 5"
output = tl_model.generate(prompt, max_new_tokens=15, **generate_kwargs)
print(output)

  0%|          | 0/15 [00:00<?, ?it/s]

0 1 1 2 3 5 8 13 21 34 55 


In [ ]:
prompt =  "2 3 5 8"
output = tl_model.generate(prompt, max_new_tokens=15, **generate_kwargs)
print(output)

  0%|          | 0/15 [00:00<?, ?it/s]

2 3 5 8 13 21 34 55 89
